<a href="https://colab.research.google.com/github/seunghyeon98/News_RAG/blob/main/news_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchainhub
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install --upgrade openai
!pip install faiss-cpu


In [ ]:
pip install -U langchain langchainhub langchain-openai

In [ ]:
import os

# LangChain Hub API 키 설정
os.environ["LANGCHAIN_HUB_API_KEY"] = "{lang chain hub key}"


In [ ]:
# lang chain hub에 prompt push
from langchain import hub
from langchain.prompts.chat import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template(input_template)
# havi_rag repo에 prompt push
# hub.push("havi_rag", prompt, new_repo_is_public=False)

'https://smith.langchain.com/prompts/havi_rag/26f335ae?organizationId=4605a3f3-a632-567d-b8b3-2308678a8a03'

In [ ]:

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아

In [ ]:
# chunk size, chunk overlap 값 설정에 대하여

# 정보의 연속성: 뉴스 기사는 문맥의 연속성이 중요합니다. 너무 작은 청크로 나누면 문맥이 손실될 수 있고, 너무 큰 청크는 모델의 입력 한계를 초과할 수 있다.

# chunk_size: 모델의 최대 입력 길이를 고려하여 적당한 크기로 설정,  예를 들어, 2048 토큰을 기준으로 하면 청크 크기를 1000-1500 글자 정도로 설정하는 것이 좋다.
# chunk_overlap: 청크 간에 문맥이 유지되도록 중첩 부분을 설정, 일반적으로 10-20% 정도의 중첩을 설정.
# 예를 들어, chunk_size를 1000으로 설정한 경우, chunk_overlap을 100-200 정도로 설정할 수 있다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [ ]:

openai_api_key="{apikey}"

from langchain_community.embeddings import OpenAIEmbeddings
openai = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Faiss를 통해 벡터DB 설정
vectorstore = FAISS.from_documents(documents=splits, embedding=openai)

# Faiss를 통해 리트리버 생성
retriever = vectorstore.as_retriever()




In [ ]:
# 나의 langchain hub의 Prompt 불러오기
prompt = hub.pull("havi_rag")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'havi_rag', 'lc_hub_commit_hash': '26f335ae9d0e8a97462ac6bb7c244a54935bf5cbcbf3ffd1746dc6f1915753f4'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
# 랭체인 허브에 있는 프롬프트 확인
print(prompt.messages[0].prompt.template)
input_template = prompt.messages[0].prompt.template

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [ ]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


llm = ChatOpenAI(
    # openai llm 모델 설정
    model_name="gpt-4-turbo-preview",
    # temperature : 얼마나 답변을 자유롭게 하느냐,
    # 0에 가까울 수록 정확하고 일관된 응답 : 코드 작성, 기술 문서 생성 task에 쓰임
    # 1에 가까울 수록 창의적이고 다양한 응답 : 스토리 생성, 브레인스토밍 등
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
    openai_api_key = openai_api_key
)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [ ]:
# rag_chain  객체의  invoke 메서드를 사용하여 사용자의 질문을 처리한다.
single_query_answer = rag_chain.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 있습니다.

In [ ]:
print(rag_chain.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
) )

부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다.부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다.


In [ ]:
print(
    rag_chain.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

- 정부 저출생 대책으로 매달 부모 급여 지급, 0세 아이는 100만원으로 인상
- 첫만남이용권 및 아동수당 제공으로 아이 돌까지 1년 동안 총 1520만원 지원
- 지자체별 추가 지원, 예를 들어 인천시는 새로 태어난 아기에게 18살 될 때까지 1억원 지급, 광주시는 17살 될 때까지 7400만원 지급 예정- 정부 저출생 대책으로 매달 부모 급여 지급, 0세 아이는 100만원으로 인상
- 첫만남이용권 및 아동수당 제공으로 아이 돌까지 1년 동안 총 1520만원 지원
- 지자체별 추가 지원, 예를 들어 인천시는 새로 태어난 아기에게 18살 될 때까지 1억원 지급, 광주시는 17살 될 때까지 7400만원 지급 예정


# Multi Query Retriever 활용

In [ ]:
# 멀티 쿼리 리트리버 활용
mqr_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)


# 체인을 생성합니다.
rag_chain = (
    {"context": mqr_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# multiqueryretriever이 질문을 어떻게 재생성하는지 확인
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question = '부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?'
multi_query_answer = rag_chain.invoke(question)



부영그룹의 출산 지원금은 얼마인가요?

부영그룹에서 출산하는 직원에게 제공하는 혜택은 무엇이 있나요?

부영그룹 출산 지원 정책에 따른 지원 금액은 어떻게 되나요?

INFO:langchain.retrievers.multi_query:Generated queries: ['부영그룹의 출산 지원금은 얼마인가요?', '', '부영그룹에서 출산하는 직원에게 제공하는 혜택은 무엇이 있나요?', '', '부영그룹 출산 지원 정책에 따른 지원 금액은 어떻게 되나요?']


부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩 지원합니다. 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다고 밝혔습니다.

## Single Query 와 Multi Query Retriever 검색 및 응답 결과 비교

In [ ]:
print('단일 쿼리:')
print(single_query_answer)

print('멀티 쿼리:')
print(multi_query_answer)


단일 쿼리:
부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 있습니다.
멀티 쿼리:
부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩 지원합니다. 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다고 밝혔습니다.


In [ ]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(query="What does the course say about regression?")
len(unique_docs)

In [ ]:
# 컬렉션을 삭제합니다.
vectorstore.delete_collection()